<a href="https://colab.research.google.com/github/4rldur0/whyfi/blob/xeoyeon/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 초기 설정

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/RAG

/content/drive/MyDrive/Colab Notebooks/RAG


In [ ]:
!pip install transformers chromadb langchain langchain_community langchain-chroma

# 2. data 불러오기

In [ ]:
import pandas as pd
file_path = '/content/drive/MyDrive/Colab Notebooks/RAG/dataset/cleaned_word_dict.csv'
data = pd.read_csv(file_path)

## pdf 가져와서 chunk하기

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_name = # 읽어오려는 파일 경로

loader = PyPDFLoader(file_name)
pages = loader.load()
text = ""
for page in pages:
    sub = page.page_content
    text += sub

In [ ]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

# token size 기준으로 contents split
tokenizer = AutoTokenizer.from_pretrained(ENCODER)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
            tokenizer,
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            separator="\n" # default: "\n\n"
        )


documents=[] # split 한 문서들을 담기 위한 array

split_conts = text_splitter.split_text(text)
for chunk_idx, split_cont in enumerate(split_conts):
    documents.append(Document(
        page_content=split_cont,
        metadata={
            "file_name": file_name,
        },
        id=chunk_idx,
    ))
    idx+=1

vectorstore.add_documents(documents)

## website 문서 로딩하기

In [ ]:
# Load documents from web
from langchain.document_loaders import WebBaseLoader

web_loader = WebBaseLoader([
    "링크 삽입"
    ]
)

data = web_loader.load()

In [ ]:
##Split documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

all_splits = text_splitter.split_documents(data)

all_splits[0]
출처: https://rfriend.tistory.com/832 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]

# 3. chromaDB에 데이터 저장하기

In [ ]:
!pip install langchain-huggingface

In [ ]:
#기존 임베딩 결과만 가져오
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding ="dragonkue/BGE-m3-ko"
collection_name = "Chroma_Collection"
chroma_path = "/content/drive/MyDrive/Colab Notebooks/RAG" #데이터 저장 경로

# Init chromadb
embedding_func = HuggingFaceEmbeddings(model_name=embedding, encode_kwargs={'normalize_embeddings':True},)
vectorstore = Chroma(
    collection_name,
    embedding_function=embedding_func,
    persist_directory=chroma_path,
    collection_metadata={"max_size": 1000}  # 용량 설정 (1000개로 확장)
)


model.safetensors:  23%|##3       | 524M/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# 기존 컬렉션 삭제
vectorstore._client.delete_collection("Chroma_Collection")

# 새로운 컬렉션 생성
vectorstore = Chroma(
    collection_name="Chroma_Collection",
    embedding_function=embedding_func,
    persist_directory="/content/drive/MyDrive/Colab Notebooks/RAG",
    collection_metadata={"max_size": 1000}
)


In [ ]:
from langchain.docstore.document import Document # 데이터의 각 row를 document 객체로 변환하여 저장하기 위함

# vectorDB에 data 추가하는 함수
def add_data_to_vectorstore(data, vectorstore):
  for index, row in data.iterrows(): # row라는 변수에 각 행을 반복적으로 가져옴.
    text = row.get("Content","")
    metadata = row.to_dict() #행 전체를 딕셔너리 형태로 변환
    metadata["source"] = metadata.get("source", f"row_{index}")   # source 필드 추가 (기본값으로 행 번호를 사용하거나 특정 열에서 가져오기)
    document = Document(page_content=text, metadata=metadata) #cocument 객체를 생성
    vectorstore.add_texts([document.page_content],[document.metadata]) # db에 데이터 추가

add_data_to_vectorstore(data, vectorstore)

# 4. Retriever 및 프롬프트 설정

In [ ]:
# Define a retriever to search in the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) #검색 시 가장 관련성 높은 3개의 문서를 반환하라는 뜻

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# system_template="""
# You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

# You MUST answer in Korean.

# Question: {question}
# Context: {context}
# Answer:
# """
system_template = """당신은 금융 전문가로, 복잡한 금융 용어를 쉽게 설명하는 데 능숙합니다. 사용자가 금융과 관련된 용어에 대해 질문하면, 다음을 수행하세요:

Context: {context}

Question: {question}

Answer:
용어 설명: 질문에 포함된 금융 용어를 간단하고 명확한 언어로 설명하세요.
추가 정보: 사용자가 해당 개념을 더 잘 이해할 수 있도록, 실제 사례나 비유를 포함하여 설명을 보완하세요.
관련 용어: 해당 용어와 연관된 다른 금융 용어나 개념을 최대 3개까지 추천하세요."""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

# 5. gemini api 설정 및 chain 구현하기

In [ ]:
!pip install langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.3 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.74.0
    Uninstalling google-cloud-aiplatform-1.74.0:
      Successfully uninstalled google-cloud-aiplatform-1.74.0


In [ ]:
PROJECT_ID = "your_project_id"
REGION = "your_region"

from google.colab import auth
auth.authenticate_user()

In [ ]:
import vertexai
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate

vertexai.init(project = PROJECT_ID , location = REGION)

chain_type_kwargs = {
    "prompt": prompt,
    "document_variable_name": "context",  # 'context'가 documents를 받을 변수임을 명시
}
llm = VertexAI(
    temperature=0,
    model_name="gemini-pro",
    max_output_tokens=1024
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# Chain 사용 예시
question = "가동률이란?"
result = chain({"question": question})

print("Answer:", result["answer"])
print("Sources:", result["source_documents"])

<ipython-input-14-22bc5efff800>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": question})


Answer: ## 가동률이란?

가동률은 **생산설비가 얼마나 활발하게 이용되고 있는지를 나타내는 지표**입니다. 쉽게 말해, **생산 능력 대비 실제 생산량의 비율**을 의미합니다. 

예를 들어, 한 공장의 생산 능력이 하루에 100개의 제품을 생산할 수 있다고 가정해 보겠습니다. 하지만 실제로 하루에 70개의 제품만 생산했다면, 이 공장의 가동률은 70%가 됩니다. 

가동률은 경기 상황을 파악하는 데 중요한 지표입니다. 경기가 좋으면 기업들은 더 많은 제품을 생산하려고 하기 때문에 가동률이 높아지고, 반대로 경기가 나쁘면 가동률이 낮아집니다. 

## 가동률을 이해하는 데 도움이 되는 실제 사례

* **제조업**: 제조업에서는 기계와 설비를 이용하여 제품을 생산합니다. 따라서 가동률은 기계와 설비가 얼마나 활발하게 가동되고 있는지를 나타냅니다. 
* **서비스업**: 서비스업에서는 사람을 이용하여 서비스를 제공합니다. 따라서 가동률은 직원들이 얼마나 열심히 일하고 있는지를 나타냅니다. 

## 가동률과 관련된 다른 용어

* **생산 능력**: 한 기업이나 국가가 생산할 수 있는 최대의 제품 또는 서비스의 양을 의미합니다. 
* **생산량**: 한 기업이나 국가가 실제로 생산한 제품 또는 서비스의 양을 의미합니다. 
* **제조업 평균 가동률**: 제조업 전체의 평균 가동률을 의미합니다. 

## 궁금한 점이 있으시면 언제든지 질문해 주세요!
Sources: [Document(id='6dbbccaf-d26a-4200-9afa-84207b2822ca', metadata={'Content': '가동률은 사업체의 생산설비 이용도를 나타내는 지표이다. 통상 가동률은 경기상황이 좋지 않아 제품의 판매량이 감소할 경우 사업체가 생산량을 축소조정하고 불필요한 장비의 가동도 줄여나가면서 하락하게 된다. 반대로 경제상황이 호전되면 가동률은 상승한다. 구체적으로 가동률은 생산능력 대비 실제 생산실적의 비율을 말한다. 여기서 생산능력이란 사업체가 설비, 인력, 조업시간